In [ ]:
# web scraping tayara tn using beautiful soup


In [ ]:
# install required libraries
!mamba install bs4==4.10.0 -y
!mamba install html5lib==1.1 -y
!pip install lxml==4.6.4
# import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.41 requires lxml>=4.9.1, but you have lxml 4.6.4 which is incompatible.


In [ ]:


# Parent page URL PS total of 179 pages with this specific filter search
Parent_url = "https://www.tayara.tn/ads/c/V%C3%A9hicules/Voitures/?minPrice=10000&minKilom%C3%A9trage=40&page=1"
Parent_data = requests.get(Parent_url).text

# Parse the HTML content
parent_soup = BeautifulSoup(Parent_data, 'html5lib')

# List to store the URLs
parent_urls = []

# Find all article elements and extract href from anchor tags
for row in parent_soup.find("body").find_all("article"):
    # Find the first anchor tag within the article
    anchor = row.find("a")
    if anchor and 'href' in anchor.attrs:
        # Append the href attribute to the list
        parent_urls.append(anchor['href'])

# Output the list of URLs
print(len(parent_urls))


65


In [ ]:
# iterate whole website :
# List to store the URLs
parent_urls = []



# Parent page URL PS total of 179 pages with this specific filter search

for i in range (1,180):
  Parent_url = f"https://www.tayara.tn/ads/c/V%C3%A9hicules/Voitures/?minPrice=10000&minKilom%C3%A9trage=40&page={i}"
  Parent_data = requests.get(Parent_url).text
  # Parse the HTML content
  parent_soup = BeautifulSoup(Parent_data, 'html5lib')
  # Find all article elements and extract href from anchor tags
  for row in parent_soup.find("body").find_all("article"):
      # Find the first anchor tag within the article
      anchor = row.find("a")
      if anchor and 'href' in anchor.attrs:
          # Append the href attribute to the list
          parent_urls.append(anchor['href'])




# Output the list of URLs
print(len(parent_urls))


5395


In [ ]:
 # going to each page and scraping relevant data

urls = []
for page in parent_urls:
  urls.append('https://www.tayara.tn'+page)
data = []
for url in urls :
  data.append(requests.get(url).text)

soups = []

for one in data :
  soups.append(BeautifulSoup(one, 'html5lib'))

print(len(soups))

5395


In [ ]:

# Initialize an empty DataFrame with specified columns
used_cars_data = pd.DataFrame(columns=["location", "time_posted", "Mileage", "Vehicle_color", "Price",
                                       "Vehicle_condition", "engine_size", "year", "gearbox", "brand", "model", "car_body", "fuel"])

# Extract price from the first row
price_row = soups[0].find('body').find_all('data')[0]
price_col = price_row.findAll("span")
price = price_col[0].text + price_col[1].text

# Extract location and time posted
location_row = soups[0].find_all(class_="flex items-center space-x-2 mb-1")[0]
location_col = location_row.findAll('span')
content = location_col[0].text.split(',')
location = content[0]
time_posted = content[1]

# Initialize a dictionary to hold the car details
car_details = {
    "location": location,
    "time_posted": time_posted,
    "Price": price
}

# Extract additional car details
for row in soups[0].find_all(class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80"):
    col = row.findAll('span')
    if len(col) < 3:  # Ensure there are enough spans
        continue
    key = col[1].text.strip()  # Get the label
    value = col[2].text.strip()  # Get the value

    # Update the car_details dictionary based on the key
    if key == "Kilométrage":
        car_details["Mileage"] = value
    elif key == "Couleur du véhicule":
        car_details["Vehicle_color"] = value
    elif key == "Etat du véhicule":
        car_details["Vehicle_condition"] = value
    elif key == "Cylindrée":
        car_details["engine_size"] = value
    elif key == "Année":
        car_details["year"] = value
    elif key == "Boite":
        car_details["gearbox"] = value
    elif key == "Marque":
        car_details["brand"] = value
    elif key == "Modèle":
        car_details["model"] = value
    elif key == "Puissance fiscale":
        car_details["horsepower"] = value
    elif key == "Type de carrosserie":
        car_details["car_body"] = value
    elif key == "Carburant":
        car_details["fuel"] = value

# Append the car details to the DataFrame
used_cars_data = pd.concat([used_cars_data, pd.DataFrame([car_details])], ignore_index=True)

# Print the extracted details
print(used_cars_data)


  location time_posted Mileage Vehicle_color  Price Vehicle_condition  \
0   Tunis    a day ago  230000          Noir  48500  Avec kilométrage   

  engine_size  year      gearbox          brand           model car_body  \
0        1.8L  2008  Automatique  Mercedes-Benz  Classe C Coupe  Berline   

      fuel horsepower  
0  Essence         11  


In [ ]:
import pandas as pd

# Initialize an empty DataFrame with specified columns
used_cars_data = pd.DataFrame(columns=["location", "time_posted", "Mileage", "Vehicle_color", "Price",
                                       "Vehicle_condition", "engine_size", "year", "gearbox", "brand", "model", "car_body", "fuel"])

# Extract price, location, and time posted for each soup object
for soup in soups:
    # Initialize a dictionary to hold the car details with default values
    car_details = {
        "location": None,
        "time_posted": None,
        "Price": None,
        "Mileage": None,
        "Vehicle_color": None,
        "Vehicle_condition": None,
        "engine_size": None,
        "year": None,
        "gearbox": None,
        "brand": None,
        "model": None,
        "horsepower": None,
        "car_body": None,
        "fuel": None
    }

    # Extract price
    price_rows = soup.find('body').find_all('data')
    if price_rows:  # Check if there are any price rows
        price_col = price_rows[0].findAll("span")
        if len(price_col) >= 2:  # Ensure there are enough spans
            car_details["Price"] = price_col[0].text + price_col[1].text

    # Extract location and time posted
    location_rows = soup.find_all(class_="flex items-center space-x-2 mb-1")
    if location_rows:  # Check if there are any location rows
        location_col = location_rows[0].findAll('span')
        if location_col:  # Ensure there are spans available
            content = location_col[0].text.split(',')
            if len(content) >= 2:  # Ensure there are enough elements
                car_details["location"] = content[0].strip()
                car_details["time_posted"] = content[1].strip()

    # Extract additional car details
    for row in soup.find_all(class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80"):
        col = row.findAll('span')
        if len(col) < 3:  # Ensure there are enough spans
            continue
        key = col[1].text.strip()  # Get the label
        value = col[2].text.strip()  # Get the value

        # Update the car_details dictionary based on the key
        if key == "Kilométrage":
            car_details["Mileage"] = value
        elif key == "Couleur du véhicule":
            car_details["Vehicle_color"] = value
        elif key == "Etat du véhicule":
            car_details["Vehicle_condition"] = value
        elif key == "Cylindrée":
            car_details["engine_size"] = value
        elif key == "Année":
            car_details["year"] = value
        elif key == "Boite":
            car_details["gearbox"] = value
        elif key == "Marque":
            car_details["brand"] = value
        elif key == "Modèle":
            car_details["model"] = value
        elif key == "Puissance fiscale":
            car_details["horsepower"] = value
        elif key == "Type de carrosserie":
            car_details["car_body"] = value
        elif key == "Carburant":
            car_details["fuel"] = value

    # Append the car details to the DataFrame
    used_cars_data = pd.concat([used_cars_data, pd.DataFrame([car_details])], ignore_index=True)

# Print the extracted details
print(used_cars_data)


        location  time_posted  Mileage Vehicle_color   Price  \
0          Tunis    a day ago   230000          Noir   48500   
1         Ariana   3 days ago  153.000          Noir   59000   
2           Sfax   3 days ago    42000        Orange   41500   
3          Tunis   4 days ago    46000         Blanc  139000   
4     La Manouba   5 days ago    75000          Noir   45000   
...          ...          ...      ...           ...     ...   
5390        None         None     None          None    None   
5391    Monastir   a year ago    90000          Bleu   72000   
5392       Tunis   a year ago   157000          Noir   23000   
5393    Monastir   a year ago   135000          Gris   11111   
5394      Nabeul  2 years ago    93000          Gris   74800   

     Vehicle_condition engine_size  year      gearbox          brand  \
0     Avec kilométrage        1.8L  2008  Automatique  Mercedes-Benz   
1     Avec kilométrage        2.0L  2012  Automatique            Kia   
2              

In [ ]:
used_cars_data.to_csv('TN_used_cars_data.csv', index=False)